# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,27.64,72,9,5.21,MV,1730670467
1,1,coahuayana de hidalgo,18.7000,-103.6583,29.36,70,66,2.51,MX,1730670468
2,2,sumbawa besar,-8.5000,117.4333,23.84,90,32,1.91,ID,1730670469
3,3,cabedelo,-6.9811,-34.8339,26.37,73,5,6.13,BR,1730670470
4,4,cidreira,-30.1811,-50.2056,21.68,85,100,6.73,BR,1730670472


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", frame_width = 800, frame_height = 600, scale = 0.8, color = "City")

# Display the map
plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 21
max_wind_speed = 10
max_cloudiness = 0
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < max_temp) &
                                     (city_data_df['Max Temp'] > min_temp) &
                                     (city_data_df['Wind Speed'] < max_wind_speed) &
                                     (city_data_df['Cloudiness'] <= max_cloudiness)
                                     ]


# Drop any rows with null values
filtered_city_data_df = city_data_df.dropna()

# Display sample data
filtered_city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,27.64,72,9,5.21,MV,1730670467
1,1,coahuayana de hidalgo,18.7000,-103.6583,29.36,70,66,2.51,MX,1730670468
2,2,sumbawa besar,-8.5000,117.4333,23.84,90,32,1.91,ID,1730670469
3,3,cabedelo,-6.9811,-34.8339,26.37,73,5,6.13,BR,1730670470
4,4,cidreira,-30.1811,-50.2056,21.68,85,100,6.73,BR,1730670472
5,5,ust-nera,64.5667,143.2000,-25.11,100,99,2.27,RU,1730670473
6,6,la ronge,55.1001,-105.2842,3.14,87,100,4.12,CA,1730670474
7,7,tiksi,71.6872,128.8694,-24.72,100,98,1.95,RU,1730670475
8,8,alibag,18.6411,72.8792,26.41,65,7,2.42,IN,1730670476
9,9,west island,-12.1568,96.8225,25.99,89,75,6.69,CC,1730670478


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,hithadhoo,-0.6000,73.0833,27.64,72,9,5.21,MV,1730670467,
1,1,coahuayana de hidalgo,18.7000,-103.6583,29.36,70,66,2.51,MX,1730670468,
2,2,sumbawa besar,-8.5000,117.4333,23.84,90,32,1.91,ID,1730670469,
3,3,cabedelo,-6.9811,-34.8339,26.37,73,5,6.13,BR,1730670470,
4,4,cidreira,-30.1811,-50.2056,21.68,85,100,6.73,BR,1730670472,
5,5,ust-nera,64.5667,143.2000,-25.11,100,99,2.27,RU,1730670473,
6,6,la ronge,55.1001,-105.2842,3.14,87,100,4.12,CA,1730670474,
7,7,tiksi,71.6872,128.8694,-24.72,100,98,1.95,RU,1730670475,
8,8,alibag,18.6411,72.8792,26.41,65,7,2.42,IN,1730670476,
9,9,west island,-12.1568,96.8225,25.99,89,75,6.69,CC,1730670478,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = { "categories": categories,
          "limit": limit,
            "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary & convert to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hithadhoo - nearest hotel: Pebbles Inn
coahuayana de hidalgo - nearest hotel: No hotel found
sumbawa besar - nearest hotel: Hotel Mekarsari
cabedelo - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
ust-nera - nearest hotel: гостиница "Солнечная"
la ronge - nearest hotel: La Ronge Hotel and Suites
tiksi - nearest hotel: Арктика
alibag - nearest hotel: Chirag executive
west island - nearest hotel: Cocos Village Bungalows
remire-montjoly - nearest hotel: Complexe Belova
bethel - nearest hotel: Hampton Inn Danbury
kunming - nearest hotel: 震庄迎宾馆
katabu - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
fisterra - nearest hotel: Hotel Temático do Banco Azul
machachi - nearest hotel: Valle Azul
hermanus - nearest hotel: Aloe guest house
georgetown - nearest hotel: Page 63 hostel
turbat - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
wagar - nearest hotel: No hotel found
blackmans bay - nearest hotel: Vi

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,hithadhoo,-0.6000,73.0833,27.64,72,9,5.21,MV,1730670467,Pebbles Inn
1,1,coahuayana de hidalgo,18.7000,-103.6583,29.36,70,66,2.51,MX,1730670468,No hotel found
2,2,sumbawa besar,-8.5000,117.4333,23.84,90,32,1.91,ID,1730670469,Hotel Mekarsari
3,3,cabedelo,-6.9811,-34.8339,26.37,73,5,6.13,BR,1730670470,No hotel found
4,4,cidreira,-30.1811,-50.2056,21.68,85,100,6.73,BR,1730670472,Hotel Castelo
...,...,...,...,...,...,...,...,...,...,...,...
580,580,taupo,-38.6833,176.0833,9.53,62,13,0.72,NZ,1730670926,Suncourt Hotel and Conference Centre
581,581,timbuktu,16.7735,-3.0074,28.81,14,23,4.59,ML,1730671189,Hotel La Maison
582,582,richards bay,-28.7830,32.0377,22.08,90,100,2.80,ZA,1730671190,SUN1 Richards Bay
583,583,ried im innkreis,48.2112,13.4886,4.04,92,47,1.34,AT,1730671192,Der Kaiserhof


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_plot_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, scale = 0.8, color="City", hover_cols=['City', 'Hotel Name', 'Country'])

# Display the map
hotel_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)